In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
batchsize = 8

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                rotation_range=0.2, 
                                shear_range=0.2, 
                                zoom_range=0.2, 
                                width_shift_range=0.2, 
                                height_shift_range=0.2, 
                                validation_split=0.2)

In [5]:
train_data = train_datagen.flow_from_directory(r"C:\Users\india\Jupyter Programs\Drowsiness\Prepared Data\train", 
                                              target_size=(80,80), batch_size=batchsize, class_mode='categorical', subset='training')
validation_data = train_datagen.flow_from_directory(r"C:\Users\india\Jupyter Programs\Drowsiness\Prepared Data\train", 
                                              target_size=(80,80), batch_size=batchsize, class_mode='categorical', subset='validation')

Found 64719 images belonging to 2 classes.
Found 16179 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r"C:\Users\india\Jupyter Programs\Drowsiness\Prepared Data\test", 
                                              target_size=(80,80), batch_size=batchsize, class_mode='categorical')

Found 4000 images belonging to 2 classes.


In [7]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [8]:
bmodel.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 80, 80, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 39, 39, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 39, 39, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 39, 39, 32)           0         ['batch_normalizati

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [10]:
checkpoint = ModelCheckpoint(r'C:\Users\india\Jupyter Programs\Drowsiness\models\model.h5', 
                             monitor='val_loss', save_best_only=True, verbose=3)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3, )

callbacks = [checkpoint, earlystop, learning_rate]

In [12]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_data, steps_per_epoch=train_data.samples//batchsize,
                    validation_data=validation_data,
                    validation_steps=validation_data.samples//batchsize,
                    callbacks=callbacks,
                    epochs=1)

C:\Users\india\AppData\Local\Temp\ipykernel_1720\3034606534.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data, steps_per_epoch=train_data.samples//batchsize,


8089/8089 [==============================] - ETA: 0s - loss: 0.1737 - accuracy: 0.9332
Epoch 1: val_loss did not improve from 0.17781
8089/8089 [==============================] - 817s 100ms/step - loss: 0.1737 - accuracy: 0.9332 - val_loss: 0.1872 - val_accuracy: 0.9220 - lr: 0.0010


Model Evaluation

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)